In [ ]:
import pandas as pd
df_mvae_1 = pd.read_csv('../outputs/sweep_outputs/news.csv').rename(columns=(lambda s: s + '_news'))
df_mvae_2 = pd.read_csv('../outputs/sweep_outputs/synthetic1.csv').rename(columns=(lambda s: s + '_synthetic1'))
df_mvae = pd.concat([df_mvae_1, df_mvae_2])
df_mvae = df_mvae.rename(columns={'{}_{}_metric/{}_{}'.format(dataset, metric_type, metric, news_synthetic): '{}_{}_{}'.format(metric, dataset_2, news_synthetic) for (dataset, dataset_2) in zip(['train','test'],['in','out']) for model in ['propensity'] for metric in ['propensity_mw_normalized','max_linear_mmd_att'] for metric_type in ['matching','decoder_t'] for news_synthetic in ['synthetic1','news']})


In [ ]:
# Plot U-statistics
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.rc('figure', figsize=(10,10))
fig, axs = plt.subplots(2)
for i,synthetic_news in enumerate(['synthetic1','news']):
    df_plot = df_mvae[['propensity_mw_normalized_in_{}'.format(synthetic_news), 'propensity_mw_normalized_out_{}'.format(synthetic_news)]]
    df_plot.columns = ['In-Sample','Hold-Out']
    sns.boxplot(color='gray', orient='h', data=df_plot, ax=axs[i])
    axs[i].set_yticklabels(axs[i].get_yticklabels(),size=10)
    axs[i].set_xlabel(None if i==0 else 'Normalised Mann-Whitney U-statistic')
fig.tight_layout()
plt.show()

In [ ]:
# See maximal imbalance depending on U-statistic decile

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.rc('figure', figsize=(10,15))
fig, axs = plt.subplots(2,2)
for i,synthetic_news in enumerate(['synthetic1','news']):
    for j,dataset in enumerate(['in','out']):
        bins = np.arange(0, 1.01, 0.1)
        df_mvae['propensity_mw_normalized_{}_{}_bin'.format(dataset, synthetic_news)] = ''
        for mini, maxi in zip(bins[:-1], bins[1:]):
            mini = np.round(mini,2)
            maxi = np.round(maxi,2)
            df_mvae['propensity_mw_normalized_{}_{}_bin'.format(dataset, synthetic_news)][(mini <= df_mvae['propensity_mw_normalized_{}_{}'.format(dataset, synthetic_news)]) & (df_mvae['propensity_mw_normalized_{}_{}'.format(dataset, synthetic_news)] < maxi)] = '${} \leq U < {}$'.format(mini,maxi)
            
        sns.boxplot(x='propensity_mw_normalized_{}_{}_bin'.format(dataset, synthetic_news), y='max_linear_mmd_att_{}_{}'.format(dataset, synthetic_news), color='gray', data=df_mvae.sort_values(by='propensity_mw_normalized_{}_{}_bin'.format(dataset, synthetic_news)), ax=axs[i,j])
        axs[i,j].set_xticklabels(axs[i,j].get_xticklabels(),rotation=90, size=10)
        axs[i,j].set_xlabel(None if i==0 else 'Normalised Mann-Whitney U-statistic')
        axs[i,j].set_ylabel(None if j>0 else 'Maximal Imbalance on MatchVAE scores')
fig.tight_layout()
plt.show()